In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import yfinance as yahoo_finance
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

def create_model(x_train, lstm_units, dropout):
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(dropout))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    model.summary()
    return model

def train_model(model, x_train, y_train, epochs, batch_size, validation_split):
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
    return model, history

def evaluate_model(model, scaler, x_test, y_test):
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(predictions - y_test)**2)
    return predictions, rmse

def plot_results(train_data, validation_data):
    plt.figure(figsize=(16,8))
    plt.title('Model Prediction')
    plt.xlabel('Date')
    plt.ylabel('Close Price USD ($)')
    plt.plot(train_data)
    plt.plot(validation_data[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    plt.show()